<a href="https://colab.research.google.com/github/pizzagatakasugi/prediction/blob/main/make_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pythonで将棋AIを作る

### インストール

In [1]:
!pip install cshogi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.6 MB/s eta 0:00:00


### パッケージインポート

In [2]:
import cshogi

## ソースコードとモジュールインストール

In [3]:
# ソースコードのクローン
!git clone https://github.com/TadaoYamaoka/python-dlshogi2.git

Cloning into 'python-dlshogi2'...
remote: Enumerating objects: 217, done.
remote: Counting objects: 100% (217/217), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 217 (delta 116), reused 212 (delta 114), pack-reused 0
Receiving objects: 100% (217/217), 313.81 MiB | 19.75 MiB/s, done.
Resolving deltas: 100% (116/116), done.
Updating files: 100% (23/23), done.


In [4]:
# パッケージインストール
%cd python-dlshogi2
!pip install -e .

/content/python-dlshogi2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/python-dlshogi2
  Preparing metadata (setup.py) ... done
  Running setup.py develop for python-dlshogi2


In [5]:
# onnxruntimeをインストール
!pip install onnxruntime-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.7 MB/s eta 0:00:00


In [ ]:
!ls

checkpoints   mcts_player.bat  onnx_player.bat	python_dlshogi2.egg-info  utils
dataset5.csv  mcts_player.sh   onnx_player.sh	README.md
LICENSE       model	       pydlshogi2	setup.py


In [6]:
!pwd

/content/python-dlshogi2


In [6]:
%cd python-dlshogi2

[Errno 2] No such file or directory: 'python-dlshogi2'
/content/python-dlshogi2


In [7]:
from pydlshogi2.player.test import OnnxPlayer2
player = OnnxPlayer2()

In [8]:
from cshogi.KIF import move_to_kif
import cshogi

In [11]:
import pandas as pd
try:
    player = OnnxPlayer2()
    df = pd.read_csv("dataset5.csv")
    df["bestlist"] = 0
    df["best2list"] = 0
    df["debug_list"] = 0
    df['bestlist'] = df['bestlist'].astype('object')
    df['best2list'] = df['best2list'].astype('object')
    df['debug_list'] = df['debug_list'].astype('object')
    KIFU_TO_SQUARE_NAMES = [
                    '１一', '１二', '１三', '１四', '１五', '１六', '１七', '１八', '１九',
                    '２一', '２二', '２三', '２四', '２五', '２六', '２七', '２八', '２九',
                    '３一', '３二', '３三', '３四', '３五', '３六', '３七', '３八', '３九',
                    '４一', '４二', '４三', '４四', '４五', '４六', '４七', '４八', '４九',
                    '５一', '５二', '５三', '５四', '５五', '５六', '５七', '５八', '５九',
                    '６一', '６二', '６三', '６四', '６五', '６六', '６七', '６八', '６九',
                    '７一', '７二', '７三', '７四', '７五', '７六', '７七', '７八', '７九',
                    '８一', '８二', '８三', '８四', '８五', '８六', '８七', '８八', '８九',
                    '９一', '９二', '９三', '９四', '９五', '９六', '９七', '９八', '９九',
                ]
    KIFU_FROM_SQUARE_NAMES = [
                    '11', '12', '13', '14', '15', '16', '17', '18', '19',
                    '21', '22', '23', '24', '25', '26', '27', '28', '29',
                    '31', '32', '33', '34', '35', '36', '37', '38', '39',
                    '41', '42', '43', '44', '45', '46', '47', '48', '49',
                    '51', '52', '53', '54', '55', '56', '57', '58', '59',
                    '61', '62', '63', '64', '65', '66', '67', '68', '69',
                    '71', '72', '73', '74', '75', '76', '77', '78', '79',
                    '81', '82', '83', '84', '85', '86', '87', '88', '89',
                    '91', '92', '93', '94', '95', '96', '97', '98', '99',
                ]
    for dfnum in range(len(df)):
        s = df["sfen"].iloc[-dfnum].split("\n")
        #print(player.features)
        player.isready()
        player.halt = 30000
        movelist = []
        cnt = 0
        allmove = []
        move2 = None
        bestlist = []
        best2list = []
        debug_list = []
        temp_best = []
        temp_best2 = []
        dlist = []
        flag = 0
        flag2 = 0

        for x in range(len(s)):
            if x < 2:
                continue
            #if x > 148:
             #   break
            if len(s[x]) < 30 and flag == 0:
                #半角の指し手を全角に変換する

                temp = s[x].split()
                #print(temp)
                #print("temp",temp[1])
                num = temp[1][0] + temp[1][1]
                for y in range(len(KIFU_FROM_SQUARE_NAMES)):
                    if num == KIFU_FROM_SQUARE_NAMES[y]:
                        sq = KIFU_TO_SQUARE_NAMES[y]
                #print("sq",sq+temp[1][2:])
                #print(len(s[x+1]))
                #print(x, s[x], s[x+1])
                if s[x].split()[1] not in ["投了" , "千日手" , "持将棋" , "反則勝ち"]:
                    movelist.append(sq+temp[1][2:])
                else:
                    movelist.append(s[x].split()[1])
                    flag = 1


            if len(s[x]) > 30 and flag2 == 0:
                #move2 = None
                #print(x)
                if cnt <=10:
                  player.halt = 10000
                board = cshogi.Board(sfen=str(s[x]))
                temp_best = []
                temp_best2 = []

                #実際に指した手をバイト文字列で取得する
                for move_s in board.legal_moves:
                    move_temp = board.move_from_usi(cshogi.move_to_usi(move_s))
                    if movelist[cnt] == move_to_kif(move_temp):
                        break
                #print("turn =", board.turn)
                #print("move =",cshogi.move_to_usi(move_s),movelist[cnt])
                move2 = move_s
                move3 = move_s
                #player.root_board = board
                allmove.append(str(cshogi.move_to_usi(move_s)))
                #print(allmove)
                player.position('startpos',allmove)
                """
                with open("sample.txt", mode="a") as f1:
                    try:
                        f1.write(str(x))
                        f1.write(" ")
                        f1.writelines(player.go())
                        f1.write("\n")
                    except TypeError as e:
                        print(e)
                        f1.write("\n")
                """
                bestmove, pondermove, cp, pv, dlist, best2move, ponder2move, cp2, pv2= player.go()
                #print(bestmove)
                if pv:
                    pvlist = pv.split()
                if pv2:
                    pv2list = pv2.split()
                #print(pvlist)
                #board = cshogi.Board()
                if movelist[cnt] not in ["投了" , "千日手" , "持将棋" , "反則勝ち"]:
                    board = cshogi.Board(sfen=str(s[x+1]))
                else:
                    flag2 = 1
                    board = cshogi.Board(sfen=str(s[x]))


                # 合法手のリストを作成
                """
                te = []
                for z in board.legal_moves:
                    te.append(cshogi.move_to_usi(z))
                #print("legalmove",te)
                """

                # 合法手を表示
                """
                for x in te:
                    move = board.move_from_usi(str(x))
                    pred = move_to_kif(move,move2)
                    if pred == "１一(11)":
                        continue
                    #print(x,pred)
                """

                #debug_info
                """
                for z in dlist:
                    #print(z["move"])
                    pred = move_to_kif(board.move_from_usi(z["move"]),move2)
                    if pred == "１一(11)":
                        continue
                    print(pred,"move_count: " ,z["move_count"],"nn_rate: ",z["nn_rate"],"win_rate: ",z["win_rate"])
                """

                """
                if board.turn == 1:
                        board_b.push_usi(str(move_s))
                        board = board_b
                """

                board2 = board.copy()
                #予測手の表示
                if pv:
                    if pv != "resign" or pv != "win":
                        if cp == "mate1":
                            temp_best.append("1手詰め"+bestmove)
                        elif cp2 == "mate3":
                            temp_best.append("3手詰め"+bestmove)
                        else:
                            for x in pvlist:
                                move = board.move_from_usi(str(x))
                                #print(move)
                                pred = move_to_kif(move,move2)
                                if pred == "１一(11)":
                                    continue
                                #print("pred1",pred)
                                temp_best.append(pred.replace("\u3000",""))
                                board.push_usi(str(x))
                                move2 = move
                    else:
                        temp_best.append(pv)

                #print("pv2:",pv2)
                #print("pv2list:",pv2list)
                #次善手の表示
                if pv2:
                    for x in pv2list:
                        move = board2.move_from_usi(str(x))
                        #print(move)
                        pred = move_to_kif(move,move3)
                        if pred == "１一(11)":
                            continue
                        #print(pred)
                        temp_best2.append(pred.replace("\u3000",""))
                        board2.push_usi(str(x))
                        move3 = move


                bestlist.append(temp_best)
                best2list.append(temp_best2)
                debug_list.append(dlist)
                #print(dlist)
                #del player
                #del board
                #gc.collect()
                #player = OnnxPlayer()
                #player.isready()
                #board_b = board
                cnt += 1

        print(len(bestlist),len(best2list),len(debug_list))
        df.at[-dfnum,"bestlist"] = bestlist
        df.at[-dfnum,"best2list"] = best2list
        df.at[-dfnum,"debug_list"] = debug_list
except Exception as e:
    print(e)
    print("dfnum=",-dfnum)
    df.to_csv("dataset6.csv", index_label=False)


/content/python-dlshogi2/pydlshogi2/player/mc_p2.py:616: RuntimeWarning: invalid value encountered in true_divide
  best2value = current_node.child_sum_value[selected2_index] / current_node.child_move_count[selected2_index]
/content/python-dlshogi2/pydlshogi2/player/mc_p2.py:616: RuntimeWarning: invalid value encountered in true_divide
  best2value = current_node.child_sum_value[selected2_index] / current_node.child_move_count[selected2_index]


KeyboardInterrupt: ignored

In [ ]:
df["bestlist"].iloc[5]

In [ ]:
df.columns

Index(['gameID', 'recordID', 'start_time', 'end_time', 'game_type',
       'battle_type', 'allotted_time', 'seconds_time', 'consumption_time',
       'place', 'appendix', 'league_results', 'end_type', 'precedence_age',
       'follower_age', 'hurigoma', 'timing_method', 'pre_adding_time',
       'fow_adding_time', 'Handicap', 'precedence_name', 'follower_name',
       'lunch_break', 'dinner_break', 'pre_lunch_time', 'pre_omit_name',
       'fow_omit_name', 'kif', 'comment', 'follwer_age', 'sfen', 'bestlist',
       'best2list', 'debug_list'],
      dtype='object')

In [ ]:
bestlist

[['１四歩(13)', '５二玉(51)'],
 ['２六歩(27)', '４四歩(43)', '７八金(69)', '４四歩(43)'],
 ['４四歩(43)', '２五歩(26)', '９四歩(93)', '６八玉(59)'],
 ['２五歩(26)', '５八金(49)'],
 ['３二銀(31)', '２五歩(26)', '４二銀(31)', '５八金(49)'],
 ['５八金(49)', '３五歩(34)', '６八玉(59)', '４二銀(31)'],
 ['３三角(22)', '３六歩(37)', '９四歩(93)', '６二玉(51)'],
 ['３六歩(37)', '９四歩(93)', '７八銀(79)', '４二銀(31)'],
 ['４二銀(31)', '９六歩(97)', '４二銀(31)', '９六歩(97)'],
 ['５八金(49)', '９四歩(93)', '５八金(49)', '９四歩(93)'],
 ['９四歩(93)', '９六歩(97)', '４二銀(31)', '４二銀(31)', '９六歩(97)', '７二玉(62)'],
 ['４六歩(47)', '９四歩(93)', '５八金(49)', '９四歩(93)'],
 ['９四歩(93)', '９六歩(97)', '４二銀(31)', '８二玉(72)', '７七玉(78)'],
 ['９六歩(97)', '９四歩(93)', '３六歩(37)', '９六歩(97)', '８二玉(72)'],
 ['９四歩(93)', '９六歩(97)', '５二金(41)', '１八香(19)'],
 ['５八金(49)', '４三銀(42)', '９六歩(97)', '９六歩(97)', '９二玉(82)'],
 ['９四歩(93)', '９六歩(97)', '４三銀(42)', '７二銀(71)', '８八玉(78)'],
 ['９六歩(97)', '９四歩(93)', '８八銀(79)', '５八金(49)', '４三銀(42)', '４四角(77)'],
 ['４三銀(42)', '９六歩(97)', '２二飛(32)', '９六歩(97)'],
 ['９六歩(97)', '９四歩(93)', '７八銀(79)', '７八金(69)', '９四歩(93)', '５五角(7

In [ ]:
best2list

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [ ]:
debug_list

[[{'move': '1c1d',
   'move_count': 1,
   'nn_rate': '0.05087',
   'win_rate': '0.48712'},
  {'move': '2c2d',
   'move_count': 0,
   'nn_rate': '0.00952',
   'win_rate': '0.00000'},
  {'move': '3c3d',
   'move_count': 1,
   'nn_rate': '0.03356',
   'win_rate': '0.50513'},
  {'move': '4c4d',
   'move_count': 0,
   'nn_rate': '0.01125',
   'win_rate': '0.00000'},
  {'move': '5c5d',
   'move_count': 1,
   'nn_rate': '0.02440',
   'win_rate': '0.46667'},
  {'move': '6c6d',
   'move_count': 1,
   'nn_rate': '0.02909',
   'win_rate': '0.46678'},
  {'move': '7c7d',
   'move_count': 1,
   'nn_rate': '0.03187',
   'win_rate': '0.45817'},
  {'move': '8c8d',
   'move_count': 1,
   'nn_rate': '0.13592',
   'win_rate': '0.50972'},
  {'move': '9c9d',
   'move_count': 1,
   'nn_rate': '0.09017',
   'win_rate': '0.49180'},
  {'move': '1a1b',
   'move_count': 0,
   'nn_rate': '0.01054',
   'win_rate': '0.00000'},
  {'move': '9a9b',
   'move_count': 0,
   'nn_rate': '0.00822',
   'win_rate': '0.00000'},

In [ ]:
df['bestlist'] = df['bestlist'].astype('object')
df.at[0,"bestlist"] = bestlist

In [ ]:
df.iat[0, -3]

[['１四歩(13)', '５二玉(51)'],
 ['２六歩(27)', '４四歩(43)', '７八金(69)', '４四歩(43)'],
 ['４四歩(43)', '２五歩(26)', '９四歩(93)', '６八玉(59)'],
 ['２五歩(26)', '５八金(49)'],
 ['３二銀(31)', '２五歩(26)', '４二銀(31)', '５八金(49)'],
 ['５八金(49)', '３五歩(34)', '６八玉(59)', '４二銀(31)'],
 ['３三角(22)', '３六歩(37)', '９四歩(93)', '６二玉(51)'],
 ['３六歩(37)', '９四歩(93)', '７八銀(79)', '４二銀(31)'],
 ['４二銀(31)', '９六歩(97)', '４二銀(31)', '９六歩(97)'],
 ['５八金(49)', '９四歩(93)', '５八金(49)', '９四歩(93)'],
 ['９四歩(93)', '９六歩(97)', '４二銀(31)', '４二銀(31)', '９六歩(97)', '７二玉(62)'],
 ['４六歩(47)', '９四歩(93)', '５八金(49)', '９四歩(93)'],
 ['９四歩(93)', '９六歩(97)', '４二銀(31)', '８二玉(72)', '７七玉(78)'],
 ['９六歩(97)', '９四歩(93)', '３六歩(37)', '９六歩(97)', '８二玉(72)'],
 ['９四歩(93)', '９六歩(97)', '５二金(41)', '１八香(19)'],
 ['５八金(49)', '４三銀(42)', '９六歩(97)', '９六歩(97)', '９二玉(82)'],
 ['９四歩(93)', '９六歩(97)', '４三銀(42)', '７二銀(71)', '８八玉(78)'],
 ['９六歩(97)', '９四歩(93)', '８八銀(79)', '５八金(49)', '４三銀(42)', '４四角(77)'],
 ['４三銀(42)', '９六歩(97)', '２二飛(32)', '９六歩(97)'],
 ['９六歩(97)', '９四歩(93)', '７八銀(79)', '７八金(69)', '９四歩(93)', '５五角(7

In [ ]:
import numpy as np
np.array(bestlist)

<ipython-input-60-c801e773d089>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(bestlist)


array([list(['１四歩(13)', '５二玉(51)']),
       list(['２六歩(27)', '４四歩(43)', '７八金(69)', '４四歩(43)']),
       list(['４四歩(43)', '２五歩(26)', '９四歩(93)', '６八玉(59)']),
       list(['２五歩(26)', '５八金(49)']),
       list(['３二銀(31)', '２五歩(26)', '４二銀(31)', '５八金(49)']),
       list(['５八金(49)', '３五歩(34)', '６八玉(59)', '４二銀(31)']),
       list(['３三角(22)', '３六歩(37)', '９四歩(93)', '６二玉(51)']),
       list(['３六歩(37)', '９四歩(93)', '７八銀(79)', '４二銀(31)']),
       list(['４二銀(31)', '９六歩(97)', '４二銀(31)', '９六歩(97)']),
       list(['５八金(49)', '９四歩(93)', '５八金(49)', '９四歩(93)']),
       list(['９四歩(93)', '９六歩(97)', '４二銀(31)', '４二銀(31)', '９六歩(97)', '７二玉(62)']),
       list(['４六歩(47)', '９四歩(93)', '５八金(49)', '９四歩(93)']),
       list(['９四歩(93)', '９六歩(97)', '４二銀(31)', '８二玉(72)', '７七玉(78)']),
       list(['９六歩(97)', '９四歩(93)', '３六歩(37)', '９六歩(97)', '８二玉(72)']),
       list(['９四歩(93)', '９六歩(97)', '５二金(41)', '１八香(19)']),
       list(['５八金(49)', '４三銀(42)', '９六歩(97)', '９六歩(97)', '９二玉(82)']),
       list(['９四歩(93)', '９六歩(97)', '４三銀(42)',